* Selenium 이 필요한 상황
* 사이트 탐색하는 법
* 요소를 선택하는 법
* 로딩을 기다리는 법
* 파일 다운로드 하는 법
* 실전

In [1]:
# 라이브러리가 이미 설치 되어있다면 생략.
!python -m pip install -q selenium

# 다음의 사이트로 접속합시다.

https://smu-momentum.github.io/crawling-study/2/iframe.html

## HTML 확인해보기

In [2]:
import requests

url = "https://smu-momentum.github.io/crawling-study/2/iframe.html"
html = requests.get(url).text

print(html)

<!DOCTYPE html>
<html>

<head>
    <title>iframe이 성가신 이유</title>
    <style>
        html,
        body,
        iframe {
            width: 100%;
            height: 100%;
            border: none;
            margin: none;
        }
    </style>
</head>

<body>
    <p style="opacity: 0;">이 태그는 투명합니다.</p>
    <p style="display: none;">이 태그는 표시되지 않습니다.</p>
    <p>이 태그는 표시됩니다.</p>
    <iframe src="https://smu-momentum.github.io/crawling-study/2/" />
</body>

</html>


# 웹 드라이버 생성하기

In [3]:
from selenium import webdriver

driver = webdriver.Chrome()

## 특정 주소로 이동하기

In [4]:
url = "https://smu-momentum.github.io/crawling-study/2/iframe.html"

driver.get(url)

## 특정 요소 1개 가져오기
= `soup.select_one()`

In [5]:
from selenium.webdriver.common.by import By

element = driver.find_element(By.CSS_SELECTOR, "body > p:nth-child(1)")
element

<selenium.webdriver.remote.webelement.WebElement (session="86af8a24fd8e28f70177c7d8a9838178", element="f.80D07DCDD319D6EF7A426A2A8C6EB79B.d.6D40744277986EDE63D19213F36E3337.e.5")>

# 특정 요소 여러 개 가져오기
= `soup.select()`

In [6]:
for tag in driver.find_elements(By.CSS_SELECTOR, "body p"):
    print(f'tag={tag.tag_name}, text={tag.text}')

tag=p, text=
tag=p, text=
tag=p, text=이 태그는 표시됩니다.


`tag.get_attribute("textContent")` 로 안 보이는 태그의 내용도 가져올 수 있다.

In [7]:
for tag in driver.find_elements(By.CSS_SELECTOR, "body p"):
    print(f'tag={tag.tag_name}, text={tag.get_attribute("textContent")}')

tag=p, text=이 태그는 투명합니다.
tag=p, text=이 태그는 표시되지 않습니다.
tag=p, text=이 태그는 표시됩니다.


iframe 안에 보이는 모멘텀 카페의 배너를 훔쳐옵시다.

In [8]:
try:
    element = driver.find_element(By.CSS_SELECTOR, "#front-cafe > a > img")
    print(element)
except Exception as exception:
    print(exception)

Message: no such element: Unable to locate element: {"method":"css selector","selector":"#front-cafe > a > img"}
  (Session info: chrome=142.0.7444.135); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#nosuchelementexception
Stacktrace:
0   chromedriver                        0x00000001052dfaa4 cxxbridge1$str$ptr + 2943380
1   chromedriver                        0x00000001052d7760 cxxbridge1$str$ptr + 2909776
2   chromedriver                        0x0000000104dee2bc _RNvCsgXDX2mvAJAg_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 74028
3   chromedriver                        0x0000000104e358ac _RNvCsgXDX2mvAJAg_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 366364
4   chromedriver                        0x0000000104e76d78 _RNvCsgXDX2mvAJAg_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 633832
5   chromedriver                        0x0000000104e29f10 _RNvCsgXDX2mvAJAg_7___rustc35___rust_no_alloc_shim_is_un

이런, iframe에 접근할 수 없어서 요소를 찾을 수 없다는 것 같아요.

`iframe` 을 찾고 해당 프레임의 HTML을 기준으로 검색할 수 있도록 할 수 있습니다.

In [9]:
iframe_element = driver.find_element(by=By.CSS_SELECTOR, value="iframe")

driver.switch_to.frame(iframe_element)

이제 검색이 됩니다.

In [10]:
img_element = driver.find_element(by=By.CSS_SELECTOR, value="#front-cafe > a > img")
img_element

<selenium.webdriver.remote.webelement.WebElement (session="86af8a24fd8e28f70177c7d8a9838178", element="f.F6797C2AADA3C41CFC997164EA3EF0D3.d.0A4DE161AFD3AA0707ED6B3305658273.e.71")>

대신 원래 있던 html의 요소들은 검색이 안되니, 돌아가고 싶다면 `driver.switch_to.default_content()` 로 돌아갈 수 있습니다.

In [11]:
driver.switch_to.default_content()

# 브라우저를 닫을 수도 있습니다.

In [12]:
driver.quit()


브라우저로 오늘의 실습 사이트에 들어가서 네이버 로그인을 해주세요.

In [13]:
driver = webdriver.Chrome()
driver.get('https://smu-momentum.github.io/crawling-study/2/')

input("로그인이 되었다면 Enter를 눌러주세요.")

''

로그인이 완료되었다면 이미지 10개를 다운로드 합시다.

다운로드 버튼을 검색해봅시다.

In [14]:
driver.find_elements(by=By.CSS_SELECTOR, value="div.se-module-file")

[]

검색이 되지 않네요. 브라우저에서 다운로드 버튼을 html 상에서 표시하고, 부모중에 iframe이 있지는 않은지 봅시다.

있는 것 같으니, 해당 iframe으로 `driver.switch_to()` 해주세요.

In [15]:
iframe = driver.find_element(by=By.CSS_SELECTOR, value="#cafe_main")
iframe

<selenium.webdriver.remote.webelement.WebElement (session="39c18756a8ba8dc9df5122a5e1a26b87", element="f.218EF184AFEFDE0D52BE116BE0E161A8.d.7A0FF2086ECBA42352AE1BFC4BDA641D.e.145")>

In [16]:
driver.switch_to.frame(iframe)

옮긴 iframe 내에서 파일 다운로드 요소들을 찾아봅시다.

In [17]:
driver.find_elements(by=By.CSS_SELECTOR, value="div.se-module-file")

[<selenium.webdriver.remote.webelement.WebElement (session="39c18756a8ba8dc9df5122a5e1a26b87", element="f.05F17A5FE8AB66351416FE85F19BB2BC.d.E7C97E708EA9108DF7198E990CF481F5.e.248")>,
 <selenium.webdriver.remote.webelement.WebElement (session="39c18756a8ba8dc9df5122a5e1a26b87", element="f.05F17A5FE8AB66351416FE85F19BB2BC.d.E7C97E708EA9108DF7198E990CF481F5.e.249")>,
 <selenium.webdriver.remote.webelement.WebElement (session="39c18756a8ba8dc9df5122a5e1a26b87", element="f.05F17A5FE8AB66351416FE85F19BB2BC.d.E7C97E708EA9108DF7198E990CF481F5.e.250")>,
 <selenium.webdriver.remote.webelement.WebElement (session="39c18756a8ba8dc9df5122a5e1a26b87", element="f.05F17A5FE8AB66351416FE85F19BB2BC.d.E7C97E708EA9108DF7198E990CF481F5.e.251")>,
 <selenium.webdriver.remote.webelement.WebElement (session="39c18756a8ba8dc9df5122a5e1a26b87", element="f.05F17A5FE8AB66351416FE85F19BB2BC.d.E7C97E708EA9108DF7198E990CF481F5.e.252")>,
 <selenium.webdriver.remote.webelement.WebElement (session="39c18756a8ba8dc9df51

In [18]:
try:
    for tag in driver.find_elements(by=By.CSS_SELECTOR, value="div.se-module-file"):
        # 다운로드 버튼을 찾고 클릭하여, "내 pc에 저장하기" 메뉴가 나타나는 것을 봅니다.
        save_button = tag.find_element(by=By.CSS_SELECTOR, value='a.se-file-save-button')
        save_button.click()

        # 다운로드 버튼을 찾고 클릭하여, "내 pc에 저장하기" 메뉴가 나타나는 것을 기다립니다.
        # 나온다면 클릭.
        local_save_button = tag.find_element(by=By.CSS_SELECTOR, value='button[data-type="local"]')
        local_save_button.click()
except Exception as exception:
    print(exception)

일부 다운로드가 누락되는 것을 알 수 있습니다.

이는 새로운 요소가 나타났다가 사라지는 것을 기다리지 못해서 발생하는 오류입니다.
다운로드 버튼이 나타나는 것을 기다렸다가 다운을 받을 수 있도록 코드에 `Wait` 기능을 넣어줍시다.

In [19]:
from selenium.webdriver.support.ui import WebDriverWait

for tag in driver.find_elements(by=By.CSS_SELECTOR, value="div.se-module-file"):
    wait = WebDriverWait(driver, timeout=10)

    # 다운로드 버튼을 찾고 클릭하여, "내 pc에 저장하기" 메뉴가 나타나는 것을 봅니다.
    wait.until(lambda _: tag.find_element(by=By.CSS_SELECTOR, value='a.se-file-save-button').is_displayed())
    tag.find_element(by=By.CSS_SELECTOR, value='a.se-file-save-button').click()

    # 다운로드 버튼을 찾고 클릭하여, "내 pc에 저장하기" 메뉴가 나타나는 것을 기다립니다.
    # 나온다면 클릭.
    wait.until(lambda _: tag.find_element(by=By.CSS_SELECTOR, value='button[data-type="local"]').is_displayed())
    tag.find_element(by=By.CSS_SELECTOR, value='button[data-type="local"]').click()